In [1]:
# import libraries
import pandas as pd

# read in teams and games
dfTeams = pd.read_excel('PD - NBA Results.xlsx', sheet_name = 0)
dfGames = pd.concat(pd.read_excel('PD - NBA Results.xlsx', sheet_name = [1,2,3,4]), ignore_index=True)

# drop unneeded fields
dfGames = dfGames[['Date','Start (ET)','Visitor/Neutral','PTS','Home/Neutral','PTS.1','Attend.']]

# filter out games that haven't been played yet
dfGames = dfGames[dfGames['PTS'] >= 0]

# create home win variable
dfGames['Visitor Win'] = dfGames['PTS'] > dfGames['PTS.1']
dfGames['Home Win'] = dfGames['PTS'] < dfGames['PTS.1']

# format date
dfGames['Date'] = pd.to_datetime(dfGames['Date'], format = '%a %b %d %Y')

# bring in conference and division
dfGames = pd.merge(dfGames, dfTeams, how = 'inner', left_on = 'Visitor/Neutral', right_on = 'Team')
dfGames = pd.merge(dfGames, dfTeams, how = 'inner', left_on = 'Home/Neutral', right_on = 'Team')

# create same conference flag
dfGames['Same Conference'] = dfGames['Conference_x'] == dfGames['Conference_y']

# create home and visitor tables
dfVisitor = dfGames[['Date','Visitor/Neutral','Visitor Win','Same Conference']]
dfVisitor.insert(1, 'Home/Visitor','Visitor')
dfVisitor = dfVisitor.rename(columns = {'Visitor/Neutral':'Team','Visitor Win':'Win'})
dfHome = dfGames[['Date','Home/Neutral','Home Win','Same Conference']]
dfHome.insert(1, 'Home/Visitor','Home')
dfHome = dfHome.rename(columns = {'Home/Neutral':'Team','Home Win':'Win'})

# union home and visitor tables
dfGames = pd.concat([dfVisitor, dfHome])

# create loss variable
dfGames['Loss'] = dfGames['Win'] == False

# bring in conference and division
dfGames = pd.merge(dfGames, dfTeams, how = 'inner', on = 'Team')

# summarize wins and losses by team
dfWinsLosses = dfGames.groupby('Team', as_index = False).agg({
    'Win':sum,
    'Loss':sum})

# change data types to integers
dfWinsLosses['Win'] = dfWinsLosses['Win'].astype('int32')
dfWinsLosses['Loss'] = dfWinsLosses['Loss'].astype('int32')

# create win pct column
dfWinsLosses['Pct'] = dfWinsLosses['Win']/(dfWinsLosses['Win']+dfWinsLosses['Loss'])

# create conference wins/losses
dfConferenceWL = dfGames[dfGames['Same Conference']==True].groupby('Team', as_index = False).agg({
    'Win':sum,
    'Loss':sum})
dfConferenceWL['Win'] = dfConferenceWL['Win'].astype('int32').astype('str')
dfConferenceWL['Loss'] = dfConferenceWL['Loss'].astype('int32').astype('str')
dfConferenceWL['Conf'] = dfConferenceWL['Win'] + '-' + dfConferenceWL['Loss']

# create home wins/losses
dfHomeWL = dfGames[dfGames['Home/Visitor']=='Home'].groupby(['Home/Visitor','Team'], as_index = False).agg({
    'Win':sum,
    'Loss':sum
})
dfHomeWL['Win'] = dfHomeWL['Win'].astype('int32').astype('str')
dfHomeWL['Loss'] = dfHomeWL['Loss'].astype('int32').astype('str')
dfHomeWL['Home'] = dfHomeWL['Win'] + '-' + dfHomeWL['Loss']

# create visitor win/losses
dfVisitorWL = dfGames[dfGames['Home/Visitor']=='Visitor'].groupby(['Home/Visitor','Team'], as_index = False).agg({
    'Win':sum,
    'Loss':sum
})
dfVisitorWL['Win'] = dfVisitorWL['Win'].astype('int32').astype('str')
dfVisitorWL['Loss'] = dfVisitorWL['Loss'].astype('int32').astype('str')
dfVisitorWL['Away'] = dfVisitorWL['Win'] + '-' + dfVisitorWL['Loss']

# sort games by date and team descending
dfGames = dfGames.sort_values(by = ['Team','Date'], ascending=False)

# create data frame containing only last 10 games by team
dfGamesL10 = dfGames.groupby('Team').head(10)

# summarize wins and losses for last 10 games by team
dfGamesL10WL = dfGamesL10.groupby('Team', as_index = False).agg({
    'Win':sum,
    'Loss':sum
})
dfGamesL10WL['Win'] = dfGamesL10WL['Win'].astype('int32').astype('str')
dfGamesL10WL['Loss'] = dfGamesL10WL['Loss'].astype('int32').astype('str')
dfGamesL10WL['L10'] = dfGamesL10WL['Win'] + '-' + dfGamesL10WL['Loss']

# reset index (dropping the index)
dfGames.reset_index(drop = True, inplace = True)

# reset index (creating column with index)
dfGames.reset_index(inplace = True)

# create data frame with most recent game by team
dfLatestGame = dfGames[['Team','Win','Loss']].groupby('Team').head(1)

# join on team, win, and loss to most recent game (to get all games that match win/loss of most recent game)
dfLatestGame = pd.merge(dfGames, dfLatestGame, how = 'inner', on = ['Team','Win','Loss'])

# create prior index column
dfLatestGame['prior index'] = dfLatestGame.groupby('Team')['index'].shift(1)

# create index diff column
dfLatestGame['index diff'] = dfLatestGame['index'] - dfLatestGame['prior index']

# fill na's with 0's
dfLatestGame = dfLatestGame.fillna(0)

# loop through games, filtering out any time index difference is greater than 1
while dfLatestGame['index diff'].max() > 1:
    dfLatestGame = dfLatestGame.fillna(0)
    dfLatestGame = dfLatestGame[dfLatestGame['index diff'] <= 1]
    
    # create prior index column
    dfLatestGame['prior index'] = dfLatestGame.groupby('Team')['index'].shift(1)
    
    # create index diff column
    dfLatestGame['index diff'] = dfLatestGame['index'] - dfLatestGame['prior index']
    
    # fill na's with 0's
    dfLatestGame = dfLatestGame.fillna(0)
    
# summarise wins and losses for latest streak
dfLatestGame = dfLatestGame.groupby('Team', as_index = False).agg({
    'Win':sum,
    'Loss':sum
})
dfLatestGame['W/L'] = 'W'
dfLatestGame.loc[dfLatestGame['Win'] < dfLatestGame['Loss'], 'W/L'] = 'L'
dfLatestGame['Strk'] = dfLatestGame['W/L'] + (dfLatestGame['Win'] + dfLatestGame['Loss']).astype('int32').astype(str)

# combine all data sets
dfSummary = pd.merge(dfTeams[['Team','Conference']], dfWinsLosses, how = 'inner', on = 'Team')
dfSummary = pd.merge(dfSummary, dfConferenceWL[['Team','Conf']], how = 'inner', on = 'Team')
dfSummary = pd.merge(dfSummary, dfHomeWL[['Team','Home']], how = 'inner', on = 'Team')
dfSummary = pd.merge(dfSummary, dfVisitorWL[['Team','Away']], how = 'inner', on = 'Team')
dfSummary = pd.merge(dfSummary, dfGamesL10WL[['Team','L10']], how = 'inner', on = 'Team')
dfSummary = pd.merge(dfSummary, dfLatestGame[['Team','Strk']], how = 'inner', on = 'Team')
dfSummary['Rank'] = dfSummary.groupby('Conference')['Pct'].rank(ascending = False).astype('int32')
dfSummary = dfSummary[['Conference','Rank','Team','Win','Loss','Pct','Conf','Home','Away','L10','Strk']]

# create two data frames for two excel sheets
dfEastern = dfSummary[dfSummary['Conference'] == 'Eastern'].drop('Conference', axis = 1)
dfWestern = dfSummary[dfSummary['Conference'] == 'Western'].drop('Conference', axis = 1)

# write to excel
with pd.ExcelWriter('2020.04.24 Preppin Data Challenge.xlsx') as writer:  
    dfEastern.to_excel(writer, sheet_name='Eastern', index = False)
    dfWestern.to_excel(writer, sheet_name='Western', index = False)